# **Laboratorio 4: Detección de Movimiento y Seguimiento de Objetos** ⚙️🖼️

**Asignatura:** Visión por Ordenador I  
**Grado:** Ingeniería Matemática e Inteligencia Artificial  
**Curso:** 2024/2025  

### **Objetivo**
En esta práctica, aprenderá a implementar algoritmos de detección de movimiento mediante sustracción de fondo y flujo óptico, y explorará el filtro de Kalman para el seguimiento de objetos.

## **Materiales**
- **Python 3.8+**
- **OpenCV**: Puede instalarlo con `pip install opencv-python`
- **Dataset de video**: Se usará un archivo de vídeo o la cámara en tiempo real para probar los métodos de detección de movimiento.

In [1]:
import cv2
import os
import numpy as np

## **Apartado A: Sustracción de Fondo**

### **Tarea A.1**: Carga de Vídeo
Cargue un vídeo en el cual se detectarán objetos en movimiento. Puede utilizar
un vídeo local o la cámara en tiempo real.

In [2]:
#TODO: Create a method that reads a video file (using VideoCapture from OpenCV) and returns its frames along with video properties
def read_video(videopath):
    """
    Reads a video file and returns its frames along with video properties.

    Args:
        videopath (str): The path to the video file.

    Returns:
        tuple: A tuple containing:
            - frames (list): A list of frames read from the video.
            - frame_width (int): The width of the video frames.
            - frame_height (int): The height of the video frames.
            - frame_rate (float): The frame rate of the video.
    """

    #TODO: Complete this line to read the video file
    cap = cv2.VideoCapture(videopath) 
    
    #TODO: Check if the video was successfully opened
    if not cap.isOpened():
        print('Error: Could not open the video file')

    #TODO: Get the szie of frames and the frame rate of the video
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # Get the width of the video frames
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # Get the height of the video frames
    frame_rate = cap.get(cv2.CAP_PROP_FPS) # Get the frame rate of the video
    
    #TODO: Use a loop to read the frames of the video and store them in a list
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()
    return frames, frame_width, frame_height, frame_rate

#TODO: Path to the video file (visiontraffic.avi)
videopath = "../data/visiontraffic.avi" 

frames, frame_width, frame_height, frame_rate = read_video(videopath)

### **Tarea A.2**: Sustracción de Fondo mediante diferencia de frames
Realice una sustracción de fondo mediante diferencia de frames, para ello guarde
un frame con el fondo estático y úselo como frame de referencia de fondo.

In [3]:
#TODO:  Show the frames to select the reference frame, press 'n' to move to the next frame and 's' to select the frame
for i, frame in enumerate(frames):
    #TODO: Show the frame
    cv2.imshow('Video', frame)
    # Wait for the key
    key = cv2.waitKey(0)
    # If the key is 'n' continue to the next frame
    if key == ord('n'):
        continue
    # If the key is 's' select the frame as the reference frame
    elif key == ord('s'):
        #TODO: Copy the frame to use it as a reference
        reference_frame = frame.copy()
        #TODO: Convert the reference frame to grayscale
        reference_frame = cv2.cvtColor(reference_frame, cv2.COLOR_BGR2GRAY)
        print('Frame {} selected as reference frame'.format(i))
        break

cv2.destroyAllWindows()

#TODO: Compute the difference between the reference frame and the rest of the frames and show the difference
for frame in frames:
    # Convert the frame to grayscale
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #TODO: Compute the difference between the reference frame and the current frame
    diff = cv2.absdiff(reference_frame, frame)
    cv2.imshow('Diferencia', diff)
    key = cv2.waitKey(1)
    if key == ord('q'):
        break

cv2.destroyAllWindows()

Frame 5 selected as reference frame


### **Tarea A.3**: Configuración de la sustracción de fondo con GMM
Configure el sustractor de fondo usando el modelo de mezcla de gaussianas
adaptativas (MOG2).

In [4]:
#TODO: Use MOG2 to detect the moving objects in the video

history = 500  # Number of frames to use to build the background model
varThreshold = 16  # Threshold to detect the background
detectShadows = True  # If True the algorithm detects the shadows

#TODO: Create the MOG2 object
mog2 = cv2.createBackgroundSubtractorMOG2(history=history, varThreshold=varThreshold, detectShadows=detectShadows)


### **Tarea A.4**: Aplicación de la Sustracción de Fondo

Aplique la sustracción de fondo en cada frame para extraer los objetos en movimiento.

In [5]:
#TODO: Use a loop to detect the moving objects in the video using the MOG2 algorithm and 
# save a video storing the parameters at the name of the file

#TODO: Create a folder to store the videos
output_folder = "output_videos"
folder_path = os.path.join(output_folder)
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

#TODO: Name of the output video file with the parameters (history, varThreshold, detectShadows)
videoname = f'output_{history}_{varThreshold}_{"shadows" if detectShadows else "no_shadows"}.avi' # Name of the output video file with the parameters
output_path = os.path.join(folder_path, videoname)


#TODO: Create a VideoWriter object to save the video
fourcc = cv2.VideoWriter_fourcc(*'XVID') # Codec to use
frame_size = (frames[0].shape[1], frames[0].shape[0]) # Size of the frames
fps = 30 # Frame rate of the video
out = cv2.VideoWriter(output_path, fourcc, fps, frame_size)

for frame in frames:
    #TODO: Apply the MOG2 algorithm to detect the moving objects
    mask = mog2.apply(frame)
    #TODO: Convert to BGR the mask to store it in the video
    mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    #TODO: Save the mask in a video
    out.write(mask)

out.release()

**Preguntas del Apartado A**
1. ¿Cómo afecta la variable `varThreshold` a la precisión de la detección?
Un valor bajo de varThreshold hace que el algoritmo detecte incluso pequeños movimientos, aumentando la sensibilidad pero también los falsos positivos. Un valor alto reduce los falsos positivos, pero podría pasar por alto movimientos sutiles.
2. ¿Qué ventajas presenta `createBackgroundSubtractorMOG2` frente a métodos simples de diferencia de imágenes?
MOG2 es más preciso y adaptable, ya que ajusta el fondo a lo largo del tiempo, detecta sombras y maneja cambios graduales en el fondo, lo que lo hace más robusto que los métodos simples, que no pueden adaptarse a cambios dinámicos o detectar sombras correctamente.

## **Apartado B: Flujo Óptico**

### **Tarea B.1**: Configuración del Flujo Óptico

Consulte la documentación de `cv2.calcOpticalFlowPyrLK` para ver qué parametros se deben definir para realizar el cálculo del flujo óptico

In [39]:
#TODO: Use the method to read the video file (slow_traffic_small.avi)
videopath = "../data/slow_traffic_small.mp4" # Path to the video file

#TODO: Define the parameters for Lucas-Kanade optical flow
winSize = (10,10)
maxLevel=3
criteria= (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)

### **Tarea B.2**: Detección de Puntos de Interés

Detecte los puntos de interés iniciales usando el algoritmo de Shi-Tomasi (`cv2.goodFeaturesToTrack`) en el primer frame.

In [41]:
#TODO: Detect the initial points of interest in the first frame
cap = cv2.VideoCapture(videopath)
ret, first_frame = cap.read()

if not ret:
    print("Failed to read the video file.")
    exit()
#TODO: Convert the first frame to grayscale
prev_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)

#TODO: Define the parameters of the Shi-Tomasi algorithm
mask = None
maxCorners = 100
qualityLevel = 0.3
minDistance = 7
blockSize = 7

# Use the function goodFeaturesToTrack to detect the points of interest
p0 = cv2.goodFeaturesToTrack(prev_gray, mask=mask, maxCorners=maxCorners, qualityLevel=qualityLevel, minDistance=minDistance, blockSize=blockSize)
if p0 is not None:
    for point in p0:
        x, y = point.ravel().astype(int)
        cv2.circle(first_frame, (x, y), 5, (0, 255, 0), -1)  # Dibuja un círculo en cada punto de interés

    # Muestra el frame con los puntos de interés dibujados
    cv2.imshow('Puntos de Interés', first_frame)
    cv2.waitKey(0)  # Espera a que se presione una te cla
    cv2.destroyAllWindows()
else:
    print("No se detectaron puntos de interés.")

# Libera el recurso de video
cap.release()

### **Tarea B.3**: Cálculo y Visualización del Flujo Óptico

In [42]:
#TODO: Use a loop to track the points of interest in the rest of the frames

cap = cv2.VideoCapture(videopath)

# List to store frames
frames = []

# Read the video frame by frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # Stop when there are no more frames

    # Append the frame to the frames list
    frames.append(frame)

# Release the video capture object
cap.release()


# Create a mask image for drawing purposes
mask = np.zeros_like(first_frame)

for i, frame in enumerate(frames[1:]):
    #TODO: Copy the frame
    input_frame = frame.copy()
    # Convert the frame to grayscale
    frame_gray = cv2.cvtColor(input_frame, cv2.COLOR_BGR2GRAY)
    #TODO: Calculate the optical flow using the Lucas-Kanade algorithm
    p1, st, err = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray, p0,None, winSize=winSize)

    # Select the points that were successfully tracked
    good_new = p1[st == 1]
    good_old = p0[st == 1]

    # Draw the tracks
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel().astype(int)
        c, d = old.ravel().astype(int)
        input_frame = cv2.circle(input_frame, (a, b), 5, (0, 0, 255), -1)
        mask = cv2.line(mask, (a, b), (c, d), (0, 255, 0), 2)

    #TODO: Update the inputs for the next iteration
    prev_gray = frame_gray.copy() # Copy the current frame to the previous frame
    p0 = good_new.reshape(-1, 1, 2) # Update the points to track
    
    # Show the frame with the tracks
    cv2.imshow('Frame', cv2.add(input_frame, mask))
    key = cv2.waitKey(10)
    if key == ord('q'):
        break

cv2.destroyAllWindows()

**Preguntas del Apartado B**
1. ¿Qué efecto tiene el parámetro `winSize` en la precisión del flujo óptico?
Parece que no tiene influencia, aunque, teóricamente, para winSize más pequeños debería ser más susceptible al ruido.
2. ¿Cómo influye el parámetro `qualityLevel` en la función `cv2.goodFeaturesToTrack` al detectar puntos de interés?
Al aumentar el quality level se muestran menos puntos de interés. Tiene sentido ya que requiere de una calidad mayor para detectar puntos

## **Apartado C: Filtro de Kalman para Seguimiento de Objetos**

### **Tarea C.1**: Configuración del Filtro de Kalman

Inicialice el filtro de Kalman (`cv2.KalmanFilter`) con una matriz de medición y transición adecuada para un seguimiento en dos dimensiones.

In [37]:
#TODO: Use the method to read the video file (slow_traffic_small.avi)
videopath = "../data/slow_traffic_small.mp4"  # Path to the video file
frames, frame_width, frame_height, frame_rate = read_video(videopath)

#TODO: Create the Kalman filter object
kf = cv2.KalmanFilter(4, 2)
#TODO: Initialize the state of the Kalman filter
kf.measurementMatrix = np.array([[1, 0, 0, 0],
                                 [0, 1, 0, 0]], np.float32)  
kf.transitionMatrix = np.array([[1, 0, 1, 0],
                                [0, 1, 0, 1],
                                [0, 0, 1, 0],
                                [0, 0, 0, 1]], np.float32)  
kf.processNoiseCov =  np.array([[1, 0, 0, 0],
                               [0, 1, 0, 0],
                               [0, 0, 5, 0],
                               [0, 0, 0, 5]], np.float32) * 0.03
measurement = np.array((2, 1), np.float32)
prediction = np.zeros((2, 1), np.float32)

#TODO: Show the frames to select the initial position of the object

for i, frame in enumerate(frames):
    # Show the frame
    cv2.imshow('Frame', frame)
    # Wait for the key
    key = cv2.waitKey(0) 
    # If the key is 'n' continue to the next frame
    if key == ord('n'):
        continue
    # If the key is 's' select the position of the object
    elif key == ord('s'):
        # Select the position of the object
        x, y, w, h = cv2.selectROI('Frame', frame, False)
        track_window = (x, y, w, h)
        #TODO: Compute the center of the object
        cx = x + w // 2
        cy = y + h // 2
        #TODO: Initialize the state of the Kalman filter
        kf.statePost = np.array([[cx], [cy], [0], [0]], np.float32)

        # Initialize the covariance matrix
        kf.errorCovPost = np.eye(4, dtype=np.float32)
        
        #Predict the position of the object
        prediction = kf.predict()
        
        #TODO: Update the measurement and correct the Kalman filter
        measurement = np.array([[cx], [cy]], np.float32)
        kf.correct(measurement)

        #TODO: Crop the object
        crop = frame[y:y+h, x:x+w].copy()
        #TODO: Convert the cropped object to HSV
        hsv_crop = cv2.cvtColor(crop, cv2.COLOR_BGR2HSV)
        #TODO: Compute the histogram of the cropped object (Reminder: Use only the Hue channel (0-180))
        crop_hist = cv2.calcHist([hsv_crop], [0], mask=None, histSize=[180], ranges=[0, 180])
        cv2.normalize(crop_hist, crop_hist, 0, 255, cv2.NORM_MINMAX)
        
        print(f'Initial position selected: {x}, {y}')
        break

cv2.destroyAllWindows()

Initial position selected: 155, 168


### **Tarea C.2**: Predicción y Corrección del Estado

Realice la predicción del estado y corrija la posición estimada en cada iteración.

In [38]:
#TODO: Use the Kalman filter to predict the position of the points of interest

term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 1)

for frame in frames[i:]:
    #TODO: Copy the frame 
    input_frame = frame.copy()
    #TODO: Convert the frame to HSV
    img_hsv = cv2.cvtColor(input_frame, cv2.COLOR_BGR2HSV)
    
    # Compute the back projection of the histogram
    img_bproject = cv2.calcBackProject([img_hsv], [0], crop_hist, [0, 180], 1)
    
    # Apply the mean shift algorithm to the back projection
    ret, track_window = cv2.meanShift(img_bproject, track_window, term_crit)
    x_,y_,w_,h_ = track_window
    #TODO: Compute the center of the object
    c_x = x_ + w_ // 2
    c_y = y_ + h_ // 2
    
    # Predict the position of the object
    prediction = kf.predict()

    #TODO: Update the measurement and correct the Kalman filter
    measurement = np.array([[np.float32(c_x)], [np.float32(c_y)]], np.float32)
    kf.correct(measurement)

    
    # Draw the predicted position
    cv2.circle(input_frame, (int(prediction[0][0]), int(prediction[1][0])), 5, (0, 0, 255), -1)
    cv2.circle(input_frame, (int(c_x), int(c_y)), 5, (0, 255, 0), -1)
    
    # Show the frame with the predicted position
    cv2.imshow('Frame', input_frame)
    key = cv2.waitKey(0)
    if key == ord('q'):
        break

cv2.destroyAllWindows()


**Preguntas del Apartado C**
1. ¿Cómo afecta el valor de `transitionMatrix` a la predicción en el filtro de Kalman?
El valor de transitionMatrix define cómo el estado del sistema cambia con el tiempo. En la predicción, esta matriz describe la relación entre el estado anterior y el nuevo estado predicho. Si la matriz no está bien ajustada, la predicción será imprecisa, ya que no reflejará correctamente el movimiento o las dinámicas del objeto.
2. ¿Cuál es la diferencia entre `measurementMatrix` y `transitionMatrix` en el contexto del seguimiento de objetos?

    transitionMatrix: Describe cómo el estado del objeto (por ejemplo, posición y velocidad) cambia con el tiempo, es decir, cómo se espera que se mueva el objeto.

    measurementMatrix: Relaciona el estado del objeto con las mediciones observadas (por ejemplo, las coordenadas de la posición del objeto en la imagen). Ayuda a actualizar el estado del filtro en función de las mediciones reales.

## **Ejercicio Adicional: Exploración del Modelo de Mezcla de Gaussianas (GMM)**

**Objetivo**: Investigue cómo funciona el modelo de mezcla de gaussianas (GMM) para mejorar la detección en condiciones de iluminación cambiantes.

1. Implementación del GMM: Utilice `cv2.createBackgroundSubtractorMOG()` y ajuste el parámetro `history` para observar cómo cambia la detección en función de la duración de la memoria del fondo.
2. Comparación con `MOG2`: Observe las diferencias en la sensibilidad a las sombras y los cambios de iluminación. Pruebe con vídeos que incluyan cambios graduales de iluminación y objetos que se detienen temporalmente.

**Preguntas del Ejercicio Adicional**
1. ¿Qué ventajas observa en `createBackgroundSubtractorMOG` en comparación con `createBackgroundSubtractorMOG2`?
2. ¿Cómo afecta el parámetro `history` al rendimiento de detección en escenas con objetos que aparecen y desaparecen?